In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Goal¶

The main idea of this notebook is inspired by [this work](https://www.kaggle.com/akshaypaliwal709/predicting-the-reserve-price-base-cpm) and we will calculate the CPM (cost per thousand impressions) on the [data](https://www.kaggle.com/saurav9786/real-time-advertisers-auction) provided by Sauran Anand before 21.06.2019 and try to predict this CPM using machine learning model on the targets after 21.06.2019.
We also have hidden goal of minimising MSE which will be revealed later on.

# Data Loading and Preprocessing

In [ ]:
data = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv',
                  parse_dates=['date'])
data.head(3)

First, let's calculate target variable using original formula provided by Akshai Paliwal.

In [ ]:
%%time

#calculating CPM
#calculating the value that the Advertisers Bid for the month of June
# CPM(the value which was the winning bid value) = 
#((revenue of the publisher*100)/revenue_share_percentage)/measurable_impressions)*1000

def weird_division(n, d):
    return n / d if d else 0


data['CPM'] = data.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)
data.CPM.values[-10:]

Columns used to create target variable can't be used as features and can now be dropped.

In [ ]:
data.drop(columns=['total_revenue', 'revenue_share_percent'], inplace=True)

We also should drop any rows with CPM < 0.

In [ ]:
data.drop(data[data['CPM'] < 0].index, inplace=True)

Assuming strong seasonality we will operate data as categorical feature in form of days of week.

In [ ]:
data['day_of_week'] = data['date'].dt.dayofweek
data.sample(10)

# Train/Test Splitting

In [ ]:
TERMINAL_DATE = '2019-06-22'    # Specified by hidden goal
VALIDATION_DATE = '2019-06-19'  # To create a validation dataset


X, y = data.drop(columns=['CPM', 'date']), data['CPM'] 

X_train = X[data['date'] < VALIDATION_DATE]
y_train = y[data['date'] < VALIDATION_DATE]

X_val = X[(data['date'] >= VALIDATION_DATE) & (data['date'] < TERMINAL_DATE)]
y_val = y[(data['date'] >= VALIDATION_DATE) & (data['date'] < TERMINAL_DATE)]

X_test = X[data['date'] >= TERMINAL_DATE]
y_test = y[data['date'] >= TERMINAL_DATE]

train_q95 = y_train.quantile(0.95)
val_q95 = y_val.quantile(0.95)
test_q95 = y_test.quantile(0.95)

X_train, y_train = X_train[y_train <= train_q95], y_train[y_train <= train_q95]
X_val, y_val = X_val[y_val <= val_q95], y_val[y_val <= val_q95]
X_test, y_test = X_test[y_test <= test_q95], y_test[y_test <= test_q95]

print('TOTAL:', X.shape, y.shape)
print('train:', X_train.shape, y_train.shape)
print('validation:', X_val.shape, y_val.shape)
print('test:', X_test.shape, y_test.shape)

# EDA

Only EDA wee need is to create corrplot of some ids to stare at them and posess some chakra magick.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize=(12, 8))
sns.heatmap(data=data.corr(), cmap='coolwarm', square=True)
plt.show

Whoa, that was strong sorcery. Now let's just look at the features and divide num bois from cat toys.

# Feature Selection

In [ ]:
X_train.describe()

In [ ]:
X_train.info()

In [ ]:
cat_features = list(range(0, 11)) + [14]
num_features = list(range(11, 14))

cat_features, num_features

# Model & Predictions

In [ ]:
from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
%%time

# initialize Pool
train_pool = Pool(X_train, 
                  y_train,
                  cat_features=cat_features)
validation_pool = Pool(X_val, 
                  y_val,
                  cat_features=cat_features)
test_pool = Pool(X_test,
                 cat_features=cat_features) 

# specify the training parameters 
model = CatBoostRegressor(cat_features=cat_features,
                          verbose=100,
                          random_seed=2020)
#train the model
model.fit(train_pool,
          eval_set=validation_pool)

# make the prediction using the resulting model
preds = model.predict(test_pool)
print(preds)

In [ ]:
test_mse = mean_squared_error(y_test, preds)
print(f'TEST MSE: {test_mse:.2f}')
if test_mse < 4850:
    print(f'{test_mse} < 4850\nHidden goal is achieved! HOCHU ZACHET!')